In [4]:
import openai
import json
import numpy as np
import os
import re
from dotenv import load_dotenv
import time
from tqdm import tqdm
load_dotenv()

MY_OPENAI_KEY = os.getenv("OPENAI_API_KEY")
client = openai.OpenAI(api_key=MY_OPENAI_KEY)

# JSON 파일 로드 함수
def load_json(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        book_data = json.load(f)
    return book_data


def get_embedding(text, model="text-embedding-ada-002", retries=3):
    for attempt in range(retries):
        try:
            response = client.embeddings.create(
                input=text,
                model=model
            )
            return response.data[0].embedding  # 최신 버전에서 데이터 접근 방식 변경됨
        except openai.APIConnectionError:
            print(f"⚠️ API 연결 오류 발생. {attempt+1}/{retries} 재시도 중...")
            time.sleep(2)  # 2초 대기 후 재시도
        except openai.APIError as e:
            print(f"❌ OpenAI API 오류: {e}")
            break
    return None


file_path = "../data/scraping/llm_final_output.json" 
embedding_save_path = "../data/scraping/book_embeddings.json" 


def embed_book_sentence(book_data):
    """
    책 데이터의 소개글을 임베딩합니다.
    """

    sentence = [book["sentence"] for book in book_data]
    hashtags = [
        [tag.replace("_", "") for tag in re.findall(r'#([\w가-힣_]+)', book["hashtags"])]
        for book in book_data
    ]
    letter = [book["letter"] for book in book_data]

    print(f"{len(sentence) = }")
    print(f"{len(hashtags) = }")
    print(f"len(letter): {len(letter)}")
    print(f"hashtags[0]: {hashtags[5]}")

    book_content = [
    f"{message} {letters} {' '.join(tags)}" for message, letters, tags in zip(sentence, letter, hashtags)
    ]

    print(len(book_content))
    print(book_content[5])

    embeddings = []
    for idx, text in enumerate(book_content):
        single_embedding = get_embedding(text)
        print(f"{idx} has completed")
        embeddings.append(single_embedding)

    return embeddings

# def save_embeddings(book_data, embeddings, save_path):
#     """
#     책 데이터와 임베딩 결과를 저장합니다.
#     """
#     ids = [book["isbn"] for book in book_data]
#     np.savez_compressed(save_path, ids=ids, embeddings=embeddings)

def save_embeddings_json(book_data, embeddings, save_path):
    """
    책 데이터와 임베딩을 JSON 형식으로 저장
    """
    data_to_save = {}

    for idx, (book, embedding) in enumerate(zip(book_data, embeddings)):
        if embedding is not None:  # None 값 필터링
            data_to_save[f"book{idx+1}"] = {
                "isbn": book["isbn"],  # 책의 ISBN (고유 식별자)
                "sentence": book["sentence"],  # 책 소개 문장
                "hashtags": book["hashtags"],
                "letter" : book["letter"],  # 해시태그
                "embedding": embedding  # 생성된 임베딩 벡터
            }

    # JSON 파일로 저장
    with open(save_path, "w", encoding="utf-8") as f:
        json.dump(data_to_save, f, ensure_ascii=False, indent=4)

    print(f"✅ JSON 파일 저장 완료: {save_path}")


In [5]:
book_data = load_json(file_path)
# book_data
book_data = list(book_data.values())

print(type(book_data))
print(book_data[0])


<class 'list'>
{'hashtags': '#행복의_방정식 #조용한_행복 #삶의_자세가_행복 #불행에_대한_수비력 #현명한_행복찾기 #마음의_여백 #긍정의_힘 #합리적_긍정 #위로의_기술 #어른의_행복', 'letter': '지금, 당신의 마음이 지옥 같다면, 이 책을 읽어보세요. 실패한 것 같은 날일수록 웃고, 감사하고, 인사하세요. 나를, 내 마음을 지키기 위해서요. 당신의 조용한 행복을 찾아보세요.', 'isbn': 9791169851053, 'sentence': '나 자신에게도 섬세함을 허락하세요. 행복은 울 일이 없는 상태, 나쁜 일이 없는 하루입니다.'}


In [94]:
# 책 소개글을 임베딩합니다.
print("Embedding book content...")
embeddings = embed_book_sentence(book_data)

# 임베딩 결과를 저장합니다.
print(f"Saving embeddings to {embedding_save_path}...")
save_embeddings_json(book_data, embeddings, embedding_save_path)

print("Embedding and saving complete.")

Embedding book content...
len(sentence) = 1781
len(hashtags) = 1781
len(letter): 1781
hashtags[0]: ['찌그러져도동그라미입니다', '김창완에세이', '삶의자세', '일상의희망', '따뜻한위로', '어른의목소리', '가벼운태도', '현재진행형', '김창완음악', '영원한아저씨']
1781
일상은 찌그러진 동그라미, 그저 오늘에 감사하며 내일을 기다립시다. "찌그러진 동그라미처럼, 우리의 일상도 때로는 울퉁불퉁하죠. 하지만 그런 날들도 결국엔 하나의 동그라미입니다. 이 책이 당신의 일상에도 따스한 온기를 전하길 바랍니다." 찌그러져도동그라미입니다 김창완에세이 삶의자세 일상의희망 따뜻한위로 어른의목소리 가벼운태도 현재진행형 김창완음악 영원한아저씨


1it [00:00,  3.41it/s]

0 has completed


2it [00:00,  3.76it/s]

1 has completed


...

1780it [13:20,  2.40it/s]

1779 has completed


1781it [13:20,  2.22it/s]

1780 has completed
Saving embeddings to ../data/scraping/book_embeddings.json...


✅ JSON 파일 저장 완료: ../data/scraping/book_embeddings.json
Embedding and saving complete.
